In [1]:
"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the Bing grounding tool from
    the Azure Agents service using a synchronous client.

USAGE:
    python sample_agents_bing_grounding.py

    Before running the sample:

    pip install azure-ai-agents azure-identity

    Set these environment variables with your own values:
    1) PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                          page of your Azure AI Foundry portal.
    2) MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_BING_CONNECTION_ID - The ID of the Bing connection, in the format of:
       /subscriptions/{subscription-id}/resourceGroups/{resource-group-name}/providers/Microsoft.MachineLearningServices/workspaces/{workspace-name}/connections/{connection-name}
"""

from json import load
import os
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import MessageRole, BingGroundingTool
from azure.identity import DefaultAzureCredential

load_dotenv = True
if load_dotenv:
    from dotenv import load_dotenv
    load_dotenv()


In [2]:
agents_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [3]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# print the connection ID to ensure it's set correctly
print(f"Using Bing connection ID: {conn_id}")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

Using Bing connection ID: /subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbinggrounding002


In [4]:
agent = agents_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-agent",
        instructions="You are a helpful agent",
        tools=bing.definitions,
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")


Created agent, ID: asst_7IxOoLKtiBKPg2CNHq34PNFN


In [5]:
# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_TSkPzPM0U4dkjqD5GUHdN4tv


In [6]:
# Create message to thread
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="How does wikipedia explain Euler's Identity?",
)
print(f"Created message, ID: {message.id}")


Created message, ID: msg_hRZABI5l7peEAJgMuMiISlzy


In [7]:
# Create and process agent run in thread with tools
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [8]:
# Fetch run steps to get the details of the agent run
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

Step step_6JetiIF99fJ3uO3vhM06I0X7 status: completed
Step step_BORpaZbfzQUDirOcjpmPXwSz status: completed
  Tool calls:
    Tool Call ID: call_hS5SFh6ez0k4D3cIwUojs3p8
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "Euler's Identity Wikipedia explanation")

